In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import copy

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from rosettastone.disentanglement import disentangle_cnn
from rosettastone.testing import (build_cnn_testbed,
                                  load_cifar10,
                                  test_disentanglement_cnn)

import wandb

In [6]:
wandb.login()

wandb: Currently logged in as: shlomihod (use `wandb login --relogin` to force relogin)


True

In [7]:
test_disentanglement_cnn(device)

Files already downloaded and verified
Files already downloaded and verified
Accuracy of the network on the 10000 test images: 44 %
Files already downloaded and verified
Files already downloaded and verified


In [8]:
net = build_cnn_testbed(True, verbose=False, device=device)

Files already downloaded and verified
Files already downloaded and verified
Accuracy of the network on the 10000 test images: 42 %


In [9]:
trainloader, testloader = load_cifar10()

_ = disentangle_cnn(net, 'conv2', 15, 'fc1',
                    train_dataloader=trainloader,
                    first_concept_dataloader=trainloader,
                    second_concept_dataloader=trainloader,
                    test_dataloader=testloader,
                    gamma=0.01,
                    with_wandb=False,
                    device=device)

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/3 [00:00<?, ?it/s]

Loss after 00768 examples: 0.003
Loss after 01568 examples: 0.002
Loss after 02368 examples: -0.000
Loss after 03168 examples: 0.000
Loss after 03968 examples: -0.001
Loss after 04768 examples: -0.001
Loss after 05568 examples: 0.000
Loss after 06368 examples: 0.002
Loss after 07168 examples: 0.001
Loss after 07968 examples: 0.002
Loss after 08768 examples: 0.000
Loss after 09568 examples: 0.001
Loss after 10368 examples: 0.001
Loss after 11168 examples: -0.000
Loss after 11968 examples: 0.001
Loss after 12768 examples: -0.002
Loss after 13568 examples: -0.000
Loss after 14368 examples: 0.004
Loss after 15168 examples: 0.002
Loss after 15968 examples: 0.001
Loss after 16768 examples: 0.000
Loss after 17568 examples: -0.001
Loss after 18368 examples: 0.001
Loss after 19168 examples: -0.004
Loss after 19968 examples: 0.001
Loss after 20768 examples: -0.004
Loss after 21568 examples: -0.001
Loss after 22368 examples: 0.005
Loss after 23168 examples: -0.001
Loss after 23968 examples: -0.00

 33%|███▎      | 1/3 [00:29<00:59, 29.98s/it]

Loss after 50352 examples: 0.003
Loss after 51152 examples: 0.007
Loss after 51952 examples: 0.003
Loss after 52752 examples: -0.001
Loss after 53552 examples: -0.001
Loss after 54352 examples: 0.003
Loss after 55152 examples: 0.000
Loss after 55952 examples: 0.001
Loss after 56752 examples: 0.007
Loss after 57552 examples: 0.004
Loss after 58352 examples: -0.003
Loss after 59152 examples: 0.001
Loss after 59952 examples: 0.000
Loss after 60752 examples: -0.001
Loss after 61552 examples: -0.004
Loss after 62352 examples: 0.002
Loss after 63152 examples: -0.002
Loss after 63952 examples: -0.002
Loss after 64752 examples: 0.005
Loss after 65552 examples: -0.004
Loss after 66352 examples: 0.001
Loss after 67152 examples: -0.003
Loss after 67952 examples: -0.008
Loss after 68752 examples: -0.008
Loss after 69552 examples: 0.007
Loss after 70352 examples: 0.010
Loss after 71152 examples: 0.000
Loss after 71952 examples: 0.002
Loss after 72752 examples: -0.000
Loss after 73552 examples: -0.0

 67%|██████▋   | 2/3 [01:00<00:30, 30.14s/it]

Loss after 100736 examples: 0.003
Loss after 101536 examples: 0.003
Loss after 102336 examples: -0.004
Loss after 103136 examples: -0.005
Loss after 103936 examples: -0.005
Loss after 104736 examples: 0.002
Loss after 105536 examples: 0.001
Loss after 106336 examples: 0.003
Loss after 107136 examples: -0.004
Loss after 107936 examples: -0.002
Loss after 108736 examples: -0.002
Loss after 109536 examples: -0.002
Loss after 110336 examples: -0.004
Loss after 111136 examples: 0.002
Loss after 111936 examples: 0.000
Loss after 112736 examples: 0.003
Loss after 113536 examples: -0.003
Loss after 114336 examples: -0.001
Loss after 115136 examples: 0.001
Loss after 115936 examples: -0.003
Loss after 116736 examples: 0.003
Loss after 117536 examples: 0.001
Loss after 118336 examples: -0.003
Loss after 119136 examples: -0.001
Loss after 119936 examples: -0.003
Loss after 120736 examples: 0.001
Loss after 121536 examples: -0.002
Loss after 122336 examples: -0.002
Loss after 123136 examples: 0.00

100%|██████████| 3/3 [01:30<00:00, 30.19s/it]


{'test_accuracy_orig': 0.4231, 'test_accuracy_alt': 0.4229}
